## Model Components

The 5 main components of a `WideDeep` model are:

1. `Wide`
2. `DeepDense` or `DeepDenseResnet`
3. `DeepText`
4. `DeepImage`
5. `deephead`

The first 4 of them will be collected and combined by the `WideDeep` collector class, while the 5th one can be optionally added to the `WideDeep` model through its corresponding parameters: `deephead` or alternatively `head_layers`, `head_dropout` and `head_batchnorm`

### 1. Wide

The wide component is a Linear layer "plugged" into the output neuron(s)

The only particularity of our implementation is that we have implemented the linear layer via an Embedding layer plus a bias. While the implementations are equivalent, the latter is faster and far more memory efficient, since we do not need to one hot encode the categorical features. 

Let's assume we the following dataset:

In [1]:
import torch
import pandas as pd
import numpy as np

from torch import nn

In [2]:
df = pd.DataFrame({'color': ['r', 'b', 'g'], 'size': ['s', 'n', 'l']})
df.head()

,color,size
0,r,s
1,b,n
2,g,l


one hot encoded, the first observation would be

In [3]:
obs_0_oh = (np.array([1., 0., 0., 1., 0., 0.])).astype('float32')

if we simply numerically encode (label encode or `le`) the values:

In [4]:
obs_0_le = (np.array([0, 3])).astype('int64')

Note that in the functioning implementation of the package we start from 1, saving 0 for padding, i.e. unseen values. 

Now, let's see if the two implementations are equivalent

In [5]:
# we have 6 different values. Let's assume we are performing a regression, so pred_dim = 1
lin = nn.Linear(6, 1)

In [6]:
emb = nn.Embedding(6, 1) 
emb.weight = nn.Parameter(lin.weight.reshape_as(emb.weight))

In [7]:
lin(torch.tensor(obs_0_oh))

tensor([-0.0483], grad_fn=<AddBackward0>)

In [8]:
emb(torch.tensor(obs_0_le)).sum() + lin.bias

tensor([-0.0483], grad_fn=<AddBackward0>)

And this is precisely how the linear component `Wide` is implemented

In [9]:
from pytorch_widedeep.models import Wide

In [14]:
?Wide

In [10]:
wide = Wide(wide_dim=10, pred_dim=1)
wide

Wide(
  (wide_linear): Embedding(11, 1, padding_idx=0)
)

Note that even though the input dim is 10, the Embedding layer has 11 weights. Again, this is because we save 0 for padding, which is used for unseen values during the encoding process

### 2. DeepDense

There are two alternatives for the so called `deepdense` component of the model: `DeepDense` and `DeepDenseResnet`. 

`DeepDense` is comprised by a stack of dense layers that receive the embedding representation of the categorical features concatenated with numerical continuous features. For those familiar with Fastai's tabular API, DeepDense is almost identical to their tabular model, although `DeepDense` allows for more flexibility when defining the embedding dimensions. 

`DeepDenseResnet` is similar to `DeepDense` but instead of dense layers, the embedding representation of the categorical features concatenated with numerical continuous features are passed through a series of dense ResNet layers. Each basic block comprises the following operations: 

<p align="center">
  <img width="400" src="../docs/figures/resnet_block.png">
</p>


Let's have a look first to `DeepDense`:

In [11]:
from pytorch_widedeep.models import DeepDense

In [12]:
# fake dataset
X_deep = torch.cat((torch.empty(5, 4).random_(4), torch.rand(5, 1)), axis=1)
colnames = ['a', 'b', 'c', 'd', 'e']
embed_input = [(u,i,j) for u,i,j in zip(colnames[:4], [4]*4, [8]*4)]
deep_column_idx = {k:v for v,k in enumerate(colnames)}
continuous_cols = ['e']

In [13]:
?DeepDense

In [15]:
# my advice would be to not use dropout in the last layer, but I add the option because you never 
# know..there is crazy people everywhere.
deepdense = DeepDense(hidden_layers=[16,8], dropout=[0.5, 0.], batchnorm=True, deep_column_idx=deep_column_idx,
                      embed_input=embed_input, continuous_cols=continuous_cols)

In [16]:
deepdense

DeepDense(
  (embed_layers): ModuleDict(
    (emb_layer_a): Embedding(4, 8)
    (emb_layer_b): Embedding(4, 8)
    (emb_layer_c): Embedding(4, 8)
    (emb_layer_d): Embedding(4, 8)
  )
  (embed_dropout): Dropout(p=0.0, inplace=False)
  (dense): Sequential(
    (dense_layer_0): Sequential(
      (0): Linear(in_features=33, out_features=16, bias=True)
      (1): LeakyReLU(negative_slope=0.01, inplace=True)
      (2): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): Dropout(p=0.5, inplace=False)
    )
    (dense_layer_1): Sequential(
      (0): Linear(in_features=16, out_features=8, bias=True)
      (1): LeakyReLU(negative_slope=0.01, inplace=True)
      (2): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): Dropout(p=0.0, inplace=False)
    )
  )
)

In [17]:
deepdense(X_deep)

tensor([[-0.2963,  0.1335, -0.4964, -0.5258, -0.5176,  0.1538, -0.8608, -0.4373],
        [-0.5645, -1.0327, -0.4927,  1.9897,  1.9988,  0.9796, -0.1428,  1.9887],
        [ 1.9892,  1.6622, -0.5086, -0.4746, -0.4995,  1.1515, -0.0226, -0.4066],
        [-0.5641, -1.0377,  1.9996, -0.5599, -0.4995, -1.2227, -0.8489, -0.6454],
        [-0.5642,  0.2747, -0.5020, -0.4295, -0.4823, -1.0622,  1.8752, -0.4993]],
       grad_fn=<NativeBatchNormBackward>)

Let's now have a look to `DeepDenseResnet`:

In [18]:
from pytorch_widedeep.models import DeepDenseResnet

In [19]:
?DeepDenseResnet

In [20]:
deepdense = DeepDenseResnet(blocks=[16,8], dropout=0.5, deep_column_idx=deep_column_idx,
                      embed_input=embed_input, continuous_cols=continuous_cols)

In [21]:
deepdense

DeepDenseResnet(
  (embed_layers): ModuleDict(
    (emb_layer_a): Embedding(4, 8)
    (emb_layer_b): Embedding(4, 8)
    (emb_layer_c): Embedding(4, 8)
    (emb_layer_d): Embedding(4, 8)
  )
  (embed_dropout): Dropout(p=0.0, inplace=False)
  (dense_resnet): Sequential(
    (lin1): Linear(in_features=33, out_features=16, bias=True)
    (bn1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (block_0): BasicBlock(
      (lin1): Linear(in_features=16, out_features=8, bias=True)
      (bn1): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (leaky_relu): LeakyReLU(negative_slope=0.01, inplace=True)
      (dp): Dropout(p=0.5, inplace=False)
      (lin2): Linear(in_features=8, out_features=8, bias=True)
      (bn2): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (resize): Sequential(
        (0): Linear(in_features=16, out_features=8, bias=True)
        (1): BatchNorm1d(8, eps=1e-05, m

In [22]:
deepdense(X_deep)

tensor([[-2.9161e-03, -3.1644e-03, -1.1107e-02,  6.0543e-02, -3.1213e-02,
          6.1364e-01,  2.6726e+00,  1.7817e+00],
        [ 4.5037e-01,  2.9428e+00, -6.5848e-03, -8.3399e-03, -7.2518e-03,
         -1.4795e-02, -4.9701e-03,  3.4010e-01],
        [-4.9371e-03, -5.8839e-03, -7.7085e-03,  1.4384e+00,  2.6421e-01,
         -9.2843e-04,  5.8450e-01,  9.9176e-01],
        [ 1.2570e+00,  4.1550e-01, -8.3861e-03, -5.4412e-03,  9.3718e-01,
          2.2961e+00,  4.3304e-01, -1.0341e-02],
        [-9.2203e-03, -2.4535e-02,  3.3787e+00, -1.2089e-03,  2.6451e+00,
         -1.3374e-02, -3.1932e-02, -2.0795e-02]], grad_fn=<LeakyReluBackward1>)

###  3. DeepText

The `DeepText` class within the `WideDeep` package is a standard and simple stack of LSTMs on top of word embeddings. You could also add a FC-Head on top of the LSTMs. The word embeddings can be pre-trained. In the future I aim to include some simple pretrained models so that the combination between text and images is fair.  

*While I recommend using the `Wide` and `DeepDense` classes within this package when building the corresponding model components, it is very likely that the user will want to use custom text and image models. That is perfectly possible. Simply, build them and pass them as the corresponding parameters. Note that the custom models MUST return a last layer of activations (i.e. not the final prediction) so that  these activations are collected by WideDeep and combined accordingly. In  addition, the models MUST also contain an attribute `output_dim` with the size of these last layers of activations.*

Let's have a look to the `DeepText` class

In [8]:
import torch
from pytorch_widedeep.models import DeepText

In [11]:
?DeepText

In [9]:
X_text = torch.cat((torch.zeros([5,1]), torch.empty(5, 4).random_(1,4)), axis=1)

In [10]:
deeptext = DeepText(vocab_size=4, hidden_dim=4, n_layers=1, padding_idx=0, embed_dim=4)

In [11]:
deeptext

DeepText(
  (word_embed): Embedding(4, 4, padding_idx=0)
  (rnn): LSTM(4, 4, batch_first=True)
)

You could, if you wanted, add a Fully Connected Head (FC-Head) on top of it

In [12]:
deeptext = DeepText(vocab_size=4, hidden_dim=8, n_layers=1, padding_idx=0, embed_dim=4, head_layers=[8,4], 
                    head_batchnorm=True, head_dropout=[0.5, 0.5])

In [13]:
deeptext

DeepText(
  (word_embed): Embedding(4, 4, padding_idx=0)
  (rnn): LSTM(4, 8, batch_first=True)
  (texthead): Sequential(
    (dense_layer_0): Sequential(
      (0): Linear(in_features=8, out_features=4, bias=True)
      (1): LeakyReLU(negative_slope=0.01, inplace=True)
      (2): BatchNorm1d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): Dropout(p=0.5, inplace=False)
    )
  )
)

Note that since the FC-Head will receive the activations from the last hidden layer of the stack of RNNs, the corresponding dimensions must be consistent.

###  4. DeepImage

The `DeepImage` class within the `WideDeep` package is either a pretrained ResNet (18, 34, or 50. Default is 18) or a stack of CNNs, to which one can add a FC-Head. If is a pretrained ResNet, you can chose how many layers you want to defrost deep into the network with the parameter `freeze`

In [14]:
from pytorch_widedeep.models import DeepImage

In [19]:
?DeepImage

In [15]:
X_img = torch.rand((2,3,224,224))

In [16]:
deepimage = DeepImage(head_layers=[512, 64, 8])

In [17]:
deepimage

DeepImage(
  (backbone): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=Tr

In [18]:
deepimage(X_img)

tensor([[-2.2825e-03, -8.3100e-04, -8.8423e-04, -1.1084e-04,  8.8529e-02,
         -5.1577e-04,  2.8343e-01, -1.7071e-03],
        [-1.8486e-03, -8.5602e-04, -1.8552e-03,  3.6481e-01,  9.0812e-02,
         -9.6603e-04,  3.9017e-01, -2.6355e-03]], grad_fn=<LeakyReluBackward1>)

if `pretrained=False` then a stack of 4 CNNs are used

In [19]:
deepimage = DeepImage(pretrained=False, head_layers=[512, 64, 8])

In [20]:
deepimage

DeepImage(
  (backbone): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.1, inplace=True)
      (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (1): Sequential(
      (0): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (2): Sequential(
      (0): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1))
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (3): Sequential(
      (0): Conv2d(256, 512, kernel_size=(1, 1), stride=(1, 1))
      (1): BatchNorm2d(512, eps=1e-05, momentum=0.01, affine=T

###  5. deephead

Note that I do not use uppercase here. This is because, by default, the `deephead` is not defined outside `WideDeep` as the rest of the components. 

When defining the `WideDeep` model there is a parameter called `head_layers` (and the corresponding `head_dropout`, and `head_batchnorm`) that define the FC-head on top of `DeeDense`, `DeepText` and `DeepImage`. 

Of course, you could also chose to define it yourself externally and pass it using the parameter `deephead`. Have a look

In [21]:
from pytorch_widedeep.models import WideDeep

In [27]:
?WideDeep